# Stand-up comedy topic modeling across cultures
## Natural Language Processing (E1282) Research Note, Dr. Sascha Göbel
- Author: Lonny Chen (216697)
- Date: 4 January 2026

In [1]:
import pandas as pd